In [ ]:
ATLAS_API_KEY = "{PASTE-API-KEY-HERE}"

In [1]:
from ripe.atlas.cousteau import (
  Ping,
  Traceroute,
  Dns,
  AtlasSource,
  AtlasCreateRequest
)
import csv
import os
import sys
import requests
import json
import time
from datetime import timedelta, date, datetime

In [2]:
import pandas as pd
import numpy as np
from ripe.atlas.cousteau import AtlasResultsRequest
import sqlite3

In [3]:
import ipaddress

In [4]:
import requests

In [33]:
import base64
from ripe.atlas.sagan.helpers import abuf as answer_buffer

In [6]:
def get_probes_from_url(url):
    #Get request
    response = requests.get(url)
    #Load json
    json_data = json.loads(response.text)
    return json_data

In [7]:
#Read json data until 'next' field is empty.
ripe_probes_url = "https://atlas.ripe.net:443/api/v2/probes/?status=1&tags=system-ipv6-works"
probes6 = []
has_next = True
while has_next:
    probe_list = get_probes_from_url(ripe_probes_url)
    for probe in probe_list['results']:
        probes6.append(probe)
    if probe_list['next'] == None:
        has_next = False
    else: ripe_probes_url = probe_list['next']

In [8]:
len(probes6)

4421

In [9]:
#Read json data until 'next' field is empty.
ripe_probes_url = "https://atlas.ripe.net:443/api/v2/probes/?status=1&tags=system-ipv4-works"
probes4 = []
has_next = True
while has_next:
    probe_list = get_probes_from_url(ripe_probes_url)
    for probe in probe_list['results']:
        probes4.append(probe)
    if probe_list['next'] == None:
        has_next = False
    else: ripe_probes_url = probe_list['next']

In [10]:
len(probes4)

10965

In [12]:
id_list4 = []
for probe in probes4:
    id_list4.append(probe['id'])

In [13]:
id_list6 = []
for probe in probes6:
    id_list6.append(probe['id'])

In [14]:
len(id_list6)

4421

In [19]:
tmp_index = 0
for i in range(0,len(id_list4),1000):
    tmp_id_list = id_list4[tmp_index:tmp_index+1000]
    tmp_index = tmp_index + 1000

    source = AtlasSource(
        type = "probes",
        value = ','.join(str(id) for id in tmp_id_list),
        requested = len(tmp_id_list)

    )

    domain = 'google.com.'
    msm_list = []

    msm_list.append(Dns(af=4, use_probe_resolver = True, query_class= "IN", query_type= "A", query_argument = domain, spread = 60, description = "IPv4 DNS Query (UDP) for " + domain +" from probe's local resolver.", protocol="UDP"))                        
    #msm_list.append(Dns(af=6, use_probe_resolver = True, query_class= "IN", query_type= "AAAA", query_argument = domain, spread = 60, description = "IPv6 DNS Query (UDP) for " + domain +" from probe's local resolver.", protocol="UDP"))                        

    #Create request, one-off five minutes from now
    atlas_request = AtlasCreateRequest(
        start_time = datetime.utcnow() + timedelta(minutes=2),
        #stop_time=datetime.utcnow() + timedelta(days=num_days, minutes=5),
        key = ATLAS_API_KEY,
        measurements = msm_list,
        sources = [source],
        is_oneoff = True
    )

    (is_success, response) = atlas_request.create()

    if is_success:
        print("Success!")
        print(len(response['measurements']))
        #If successful write msm_id to csv
        with open('metadata/msm_id_list_all_ipv4_probes.csv', 'a') as f:
            writer = csv.writer(f)
            for j in response['measurements']:
                writer.writerows([[j]])
    else:
        print("Error!")
        print(response)






Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1


In [20]:
tmp_index = 0
for i in range(0,len(id_list6),1000):
    tmp_id_list = id_list6[tmp_index:tmp_index+1000]
    tmp_index = tmp_index + 1000

    source = AtlasSource(
        type = "probes",
        value = ','.join(str(id) for id in tmp_id_list),
        requested = len(tmp_id_list)

    )

    domain = 'google.com.'
    msm_list = []

    #msm_list.append(Dns(af=4, use_probe_resolver = True, query_class= "IN", query_type= "A", query_argument = domain, spread = 60, description = "IPv4 DNS Query (UDP) for " + domain +" from probe's local resolver.", protocol="UDP"))                        
    msm_list.append(Dns(af=6, use_probe_resolver = True, query_class= "IN", query_type= "AAAA", query_argument = domain, spread = 60, description = "IPv6 DNS Query (UDP) for " + domain +" from probe's local resolver.", protocol="UDP"))                        

    #Create request, one-off five minutes from now
    atlas_request = AtlasCreateRequest(
        start_time = datetime.utcnow() + timedelta(minutes=2),
        #stop_time=datetime.utcnow() + timedelta(days=num_days, minutes=5),
        key = ATLAS_API_KEY,
        measurements = msm_list,
        sources = [source],
        is_oneoff = True
    )

    (is_success, response) = atlas_request.create()

    if is_success:
        print("Success!")
        print(len(response['measurements']))
        #If successful write msm_id to csv
        with open('metadata/msm_id_list_all_ipv6_probes.csv', 'a') as f:
            writer = csv.writer(f)
            for j in response['measurements']:
                writer.writerows([[j]])
    else:
        print("Error!")
        print(response)







Success!
1
Success!
1
Success!
1
Success!
1
Success!
1


In [21]:
measurements4 = pd.read_csv('metadata/msm_id_list_all_ipv4_probes.csv', header=None)[0].unique()
measurements6 = pd.read_csv('metadata/msm_id_list_all_ipv6_probes.csv', header=None)[0].unique()

In [22]:
measurements4

array([28317213, 28317214, 28317215, 28317216, 28317217, 28317218,
       28317219, 28317220, 28317221, 28317222, 28317223])

In [23]:
measurements6

array([28317224, 28317225, 28317226, 28317227, 28317228])

In [24]:
msm_result = []
for msm_id in measurements4:
    print('.',end='')
    kwargs = {
        "msm_id": msm_id
    }
    is_success, results = AtlasResultsRequest(**kwargs).create()
    if is_success:
        for result in results:
            msm_result.append(result)
    else:
        print(results)

for msm_id in measurements6:
    print('.',end='')
    kwargs = {
        "msm_id": msm_id
    }
    is_success, results = AtlasResultsRequest(**kwargs).create()
    if is_success:
        for result in results:
            msm_result.append(result)
    else:
        print(results)

................

In [25]:
len(msm_result)

14812

In [26]:
def parse_result(result_dict):
    resolver_address = address_family = rt = error = abuf = ''
    
    
    #Get resolver IP
    if 'dst_addr' in result_dict:
        resolver_address = result_dict['dst_addr']
    elif 'dst_name' in result_dict:
        resolver_address = result_dict['dst_name']
    
    #Get address family
    if 'af' in result_dict:
        address_family = result_dict['af']
        
        
    #actual result dictionary inside the outer dictionary
    if 'result' in result_dict:
        if 'rt' in result_dict['result']:
            rt = str(result_dict['result']['rt'])
            
        #Abuf contains almost all additional info we need
        if 'abuf' in result_dict['result']:
            abuf = result_dict['result']['abuf']
        '''
            raw_data = abuf.AbufParser.parse(base64.b64decode(result_dict['result']['abuf']))
            #Return code
            if 'HEADER' in raw_data:
                if 'ReturnCode' in raw_data['HEADER']:
                    rcode = raw_data['HEADER']['ReturnCode']
                else:
                    rcode = 'No return code'
            #Target name
            if 'QuestionSection' in raw_data:
                if len(raw_data['QuestionSection']) > 0:
                    if 'Qname' in raw_data['QuestionSection'][0]:
                        target_name = raw_data['QuestionSection'][0]['Qname']
            #Response type, TTL and address
            if 'AnswerSection' in raw_data:
                if len(raw_data['AnswerSection']) > 0:
                    for answer in raw_data['AnswerSection']:
                        if 'Type' in answer:
                            response_type = answer['Type']
                            if response_type != 'CNAME':
                                if 'Address' in answer:
                                    response_address = answer['Address']
                                else:
                                    print('Non-CNAME with no address:' + response_type)
                        if 'TTL' in answer:
                            ttl = answer['TTL']
        '''         
            
    
    #error case, should not have result dictionary
    if 'error' in result_dict:
        if 'result' in result_dict:
            print('error and result in same dict')
        error = result_dict['error']
        if type(error) == dict:
            error = json.dumps(error)
    
    
    
    
    
    return resolver_address, address_family, rt, error, abuf

In [27]:
#df = pd.DataFrame(columns=['msm_id', 'prb_id', 'timestamp',
#                           'resolver_address', 'address_family',
#                           'rt', 'error', 'is_local_resolver'])

conn = sqlite3.connect("data/ripe-dns-all-probes-2020.db")
conn.isolation_level = None
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS "data" ("msm_id" INTEGER, "prb_id" INTEGER, "timestamp" INTEGER, "resolver_address" TEXT, "address_family" TEXT, "rt" TEXT, "error" TEXT, "local_resolver" TEXT, "abuf" TEXT);')

chunk_of_data = []

total_len = 0


for i in range(len(msm_result)):
    print('.', end='')
    msm_id = prb_id = timestamp = ''

    #Can take this from normal header
    msm_id = msm_result[i]['msm_id']
    prb_id = msm_result[i]['prb_id']
    timestamp = msm_result[i]['timestamp']

    #If the local resolver flag is set, RIPE Atlas always returns a resultset, even if the length is 1
    #If a public resolver is used, we get single results instead (the results contain the first two answers)

    is_local_resolver = 'Unknown'

    #Local resolver
    if 'resultset' in msm_result[i]:

        is_local_resolver = 'True'

        #Edge case, not sure if this even happens, since without a working local resolver,
        #something might be wrong with the probe
        #if len(results[i]['resultset']) == 1:
        #    if 'error' in results[i]['resultset'][0]:
        #        print('Local resolver with single error found')
        #        print(results[i])

        #This is mostly a sanity check, this should in theory never happen
        #for result in msm_result[i]['resultset']:
        #    if not 'submax' in result:
        #        print('submax not in resultset section of msm '+str(msm_id) + 'probe ' + str(prb_id))

        #Parse results
        for result in msm_result[i]['resultset']:
            resolver, af, rt, error, abuf = parse_result(result)
            if af == '':
                if 'from' in msm_result[i]:
                    af = '6' if msm_result[i]['from'].find(':') > 0 else '4'
                #Just to be safe:
                #if results[i]['from'].find(':') > 0:
                #    if results[i]['from'].find('.') > 0:
                #        inferred_address_family = 'Unknown'
            
            chunk_of_data.append((msm_id, prb_id, timestamp, resolver, af, rt, error, is_local_resolver, abuf))


    #Public resolver
    elif 'result' in msm_result[i]:

        is_local_resolver = 'False'

        resolver, af, rt, error, abuf = parse_result(msm_result[i])
        if af == '':
            if 'from' in msm_result[i]:
                af = '6' if msm_result[i]['from'].find(':') > 0 else '4'
        chunk_of_data.append((msm_id, prb_id, timestamp, resolver, af, rt, error, is_local_resolver, abuf))



    #No results, still want to collect failure information; seems to have info 
    #on timeouts, i.e. resolver unreachable
    else:

        is_local_resolver = 'False'

        resolver = "Failed for other reasons"

        #The dst_addr field seems to be more likely in erronous results
        if 'dst_addr' in msm_result[i]:
            resolver = msm_result[i]['dst_addr']

        #If addr doesnt work, try name; should be an IP address as well
        elif 'dst_name' in msm_result[i]:
            resolver = msm_result[i]['dst_name']

        #If af is not in there we can kind of infer it from the source IP
        af = 'Unknown'
        if 'af' in msm_result[i]:
            af = msm_result[i]['af']
        else:
            if 'from' in msm_result[i]:
                af = '6' if msm_result[i]['from'].find(':') > 0 else '4'
                #Just to be safe:
                #if results[i]['from'].find(':') > 0:
                #    if results[i]['from'].find('.') > 0:
                #        af = 'Unknown'


        response_time = ''
        abuf = 'error, no abuf'

        error = 'Unknown'
        if 'error' in msm_result[i]:
            error = msm_result[i]['error']
            
        if type(error) == dict:
            error = json.dumps(error)
        chunk_of_data.append((msm_id, prb_id, timestamp, resolver, af, rt, error, is_local_resolver, abuf))
        
    #dump data every 500 msm results
    if (i%1000 == 0 and i > 0) or i == len(msm_result)-1:
        print('\n'+str(i))
        total_len = total_len + len(chunk_of_data)
        
        print('Writing chunk')
        timer_for_chunk_tx = time.time()
        cur.execute('BEGIN TRANSACTION')
        for res_tuple in chunk_of_data:
                cur.execute('INSERT INTO "data" ("msm_id", "prb_id", "timestamp", "resolver_address", "address_family", "rt", "error", "local_resolver", "abuf") VALUES (?,?,?,?,?,?,?,?,?)', res_tuple)

        cur.execute('COMMIT')
        print('Done writing chunk')
        print('Clearing list')
        chunk_of_data.clear()
        print("Time Taken: %.3f sec" % (time.time() - timer_for_chunk_tx))

    #print(' ')
    #print('Finished msm '+str(msm))




........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Done writing chunk
Clearing list
Time Taken: 0.561 sec
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [34]:
def parse_abuf(abuf_str):
    print('.', end='')
    rcode = target_name = response_type = response_address = ttl = ''
    if abuf_str != '' and abuf_str != 'error, no abuf':
        raw_data = answer_buffer.AbufParser.parse(base64.b64decode(abuf_str))
        #Return code
        if 'HEADER' in raw_data:
            if 'ReturnCode' in raw_data['HEADER']:
                rcode = raw_data['HEADER']['ReturnCode']
            else:
                rcode = 'No return code'
        #Target name
        if 'QuestionSection' in raw_data:
            if len(raw_data['QuestionSection']) > 0:
                if 'Qname' in raw_data['QuestionSection'][0]:
                    target_name = raw_data['QuestionSection'][0]['Qname']
        #Response type, TTL and address
        if 'AnswerSection' in raw_data:
            if len(raw_data['AnswerSection']) > 0:
                for answer in raw_data['AnswerSection']:
                    if 'Type' in answer:
                        response_type = answer['Type']
                        if response_type != 'CNAME':
                            if 'Address' in answer:
                                response_address = answer['Address']
                            else:
                                print('Non-CNAME with no address:' + response_type)
                    if 'TTL' in answer:
                        ttl = answer['TTL']
    return target_name, rcode, response_type, response_address, ttl

In [29]:
conn = sqlite3.connect("data/ripe-dns-all-probes-2020.db")
df = pd.read_sql_query("SELECT * FROM data", conn)
conn.close()

In [30]:
abuf_str_list = df['abuf'].value_counts().index.tolist()
print(len(abuf_str_list))

28289


In [31]:
print(len(df))

29825


In [35]:
ctr = 0
abuf_parse_res = []
for abuf_str in abuf_str_list:
    if ctr%10000==0:
        print('\n'+str(ctr))
    target_name, rcode, response_type, response_address, ttl = parse_abuf(abuf_str)
    abuf_parse_res.append((abuf_str, target_name, rcode, response_type, response_address, ttl))
    ctr = ctr + 1
    



0
.....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
20000
....................................................................................

In [36]:
df_abuf_res = pd.DataFrame(abuf_parse_res, columns = ['abuf', 'target_name', 'rcode', 'response_type', 'response_address', 'ttl'])

df_sql = pd.merge(df, df_abuf_res, how='left', on=['abuf'])
db = sqlite3.connect("data/ripe-dns-all-probes-with-abuf-2020.db")
df_sql.to_sql('data', db, if_exists='append', index=False)
db.close()



In [37]:
msm_result.clear()

In [38]:
chunk_of_data.clear()

In [68]:
df_sql

,msm_id,prb_id,timestamp,resolver_address,address_family,rt,error,local_resolver,abuf,target_name,rcode,response_type,response_address,ttl
0,22001500,627,1560433280,193.162.153.164,4,52.577,,True,sPKBgAABAAEABAAIBmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.207.206,112
1,22001500,627,1560433280,194.239.134.83,4,53.224,,True,/KWBgAABAAEABAAIBmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.21.174,279
2,22001500,10001,1560433694,fd00::eadf:70ff:fea3:7a26,6,7.765,,True,y7yBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.168.238,85
3,22001500,10001,1560433694,192.168.178.1,4,1.259,,True,08KBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.168.238,84
4,22001500,10006,1560433693,8.8.8.8,4,28.205,,True,QpqBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.169.46,264
5,22001500,10007,1560433695,75.75.75.75,4,14.097,,True,sPKBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.6.78,247
6,22001500,10007,1560433695,2001:558:feed::1,6,13.821,,True,VkGBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.194.174,273
7,22001500,10007,1560433695,75.75.76.76,4,31.651,,True,/KWBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.195.78,240
8,22001500,10009,1560433694,192.168.2.1,4,10.52,,True,rqaBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.5.110,242
9,22001500,10009,1560433694,2601:646:c000:d16c::1,6,0.665,,True,UQKBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.5.110,241


In [39]:
def is_global_ip(ipaddr):
    try:
        return ipaddress.ip_address(ipaddr).is_global
    except (ValueError):
        return False

In [40]:
df_sql['is_global_ip'] = df_sql.apply(lambda row: is_global_ip(row.resolver_address), axis=1)



In [42]:
globally_reachable_resolvers = df_sql[df_sql.is_global_ip == True].resolver_address.value_counts().index.tolist()

In [43]:
len(globally_reachable_resolvers)

4680

In [78]:
df_sql.prb_id.nunique()

9729

In [79]:
df_sql[df_sql.is_global_ip == True].prb_id.nunique()

5500

In [46]:
dff = df_sql.copy(deep=True)

In [44]:
df_centralised_resolvers = pd.read_csv('metadata/resolvers-extended.csv')
df_centralised_resolvers

,ip,resolver_name
0,185.228.168.168,CleanBrowsing
1,1.1.1.1,Cloudflare
2,8.8.8.8,Google
3,208.67.222.123,OpenDNS
4,185.121.177.177,OpenNIC
...,...,...
106,2a02:6b8:0:1::feed:ff,Yandex
107,2a02:6b8::feed:bad,Yandex
108,2a02:6b8:0:1::feed:bad,Yandex
109,2a02:6b8::feed:a11,Yandex


In [45]:
df_centralised_resolvers.rename(columns={'ip':'resolver_address'}, inplace=True)

In [47]:
dff = pd.merge(dff, df_centralised_resolvers, how='left', on=['resolver_address'])

In [48]:
dff['resolver_name'] = dff['resolver_name'].fillna(value='local')

In [49]:
dff

,msm_id,prb_id,timestamp,resolver_address,address_family,rt,error,local_resolver,abuf,target_name,rcode,response_type,response_address,ttl,is_global_ip,resolver_name
0,28317213,1,1606922980,192.168.178.1,4,56.157,,True,ieOBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.20.110,287,False,local
1,28317213,1000,1606922980,192.168.1.1,4,163.729,,True,CM6BgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.25.142,300,False,local
2,28317213,1002,1606922979,192.168.103.1,4,94.026,,True,1DKBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,142.250.66.206,163,False,local
3,28317213,102,1606922980,193.62.22.10,4,77.568,,True,07CBgAABAAEABAAIBmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.210.46,300,True,local
4,28317213,102,1606922980,193.62.22.2,4,73.008,,True,Uj2BgAABAAEABAAIBmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.213.14,300,True,local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37449,28317228,1000095,1606923502,8.8.4.4,4,7.721,,True,K7mBgAABAAEAAAAABmdvb2dsZQNjb20AABwAAcAMABwAAQ...,google.com.,NOERROR,AAAA,2a00:1450:400e:800:0:0:0:200e,299,True,Google
37450,28317228,1000095,1606923502,8.8.4.4,4,7.721,,True,K7mBgAABAAEAAAAABmdvb2dsZQNjb20AABwAAcAMABwAAQ...,google.com.,NOERROR,AAAA,2a00:1450:400e:800:0:0:0:200e,299,True,Google
37451,28317228,1000652,1606923520,192.168.242.16,4,1.549,,True,QreBgAABAAEAAAAABmdvb2dsZQNjb20AABwAAcAMABwAAQ...,google.com.,NOERROR,AAAA,2a00:1450:400e:80d:0:0:0:200e,184,False,local
37452,28317228,1000022,1606923542,1.1.1.1,4,1.677,,True,QweBgAABAAEAAAAABmdvb2dsZQNjb20AABwAAcAMABwAAQ...,google.com.,NOERROR,AAAA,2a00:1450:4003:80a:0:0:0:200e,284,True,Cloudflare


In [50]:
def is_private_ip(ipaddr):
    try:
        return ipaddress.ip_address(ipaddr).is_private
    except (ValueError):
        return True

In [51]:
dff['is_private_ip'] = dff.apply(lambda row: is_private_ip(row.resolver_address), axis=1)




In [52]:
dff[(dff.is_global_ip == False) & (dff.is_private_ip == False)]

,msm_id,prb_id,timestamp,resolver_address,address_family,rt,error,local_resolver,abuf,target_name,rcode,response_type,response_address,ttl,is_global_ip,resolver_name,is_private_ip
1244,28317213,3960,1606922981,100.83.254.190,4,65.783,,True,ScmBgAABAAYAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,64.233.165.101,40,False,local,False
1245,28317213,3960,1606922981,100.83.254.30,4,67.03,,True,dgaBgAABAAYAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,64.233.161.100,128,False,local,False
7303,28317215,13295,1606922991,100.64.0.83,4,13.32,,True,CP+BgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.208.110,121,False,local,False
7304,28317215,13295,1606922991,100.64.0.84,4,11.263,,True,33eBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.214.14,66,False,local,False
7354,28317215,13376,1606922981,100.100.100.1,4,42.83,,True,sPKBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.16.14,300,False,local,False
7683,28317215,13914,1606922982,100.127.248.30,4,128.724,,True,e+SBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.207.14,299,False,local,False
7987,28317215,14550,1606922982,100.64.42.1,4,0.625,,True,VkGBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.6.78,259,False,local,False
8185,28317216,15039,1606922985,100.100.0.100,4,9.783,,True,WEiBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.15.78,191,False,local,False
9711,28317216,19015,1606922983,100.64.100.64,4,1.036,,True,68qBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.8.174,15,False,local,False
10375,28317217,19977,1606922986,100.64.3.1,4,3.895,,True,9XmBgAABAAYAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,209.85.233.101,40,False,local,False


In [53]:
def get_json_resource_from_absolute_uri(url, query_params):
    try: res = requests.get(url, params = query_params)
    except Exception as e: print(e, file=sys.stderr)
    else:
        try: res_json = res.json()
        except Exception as e: print(e, file=sys.stderr)
        else: 
            return res_json

In [54]:
def get_asn_from_endpoint(endpoint):
    asn = holder = None
    base_uri = 'https://stat.ripe.net'; url = '%s/data/prefix-overview/data.json'%base_uri
    params = {'resource' : endpoint}
    try: res = get_json_resource_from_absolute_uri(url, params)
    except Exception as e: print(e, file=sys.stderr); return None
    try:
        asns_holders = []
        for item in res['data']['asns']:
            asn = item['asn']; holder = item['holder']
            asns_holders.append((asn, holder))
    except Exception as e: print(e, file=sys.stderr)
    if len(asns_holders) > 1:
        print(asns_holders)
    elif len(asns_holders) == 0:
        return None
    #print(asns_holders[0][1])
    return asns_holders[0][1]

In [55]:
unid_rslv = pd.DataFrame(dff[(dff.is_global_ip == True) & (dff.resolver_name == 'local')].resolver_address.unique(), columns=['resolver_address'])

In [56]:
unid_rslv['owner'] = unid_rslv.apply(lambda row: get_asn_from_endpoint(row.resolver_address), axis=1)

Expecting value: line 1 column 1 (char 0)
'NoneType' object is not subscriptable


[(3910, 'CENTURYLINK-EUROPE-LEGACY-QWEST'), (209, 'CENTURYLINK-US-LEGACY-QWEST'), (3908, 'CENTURYLINK-ASIA-LEGACY-QWEST')]


Expecting value: line 1 column 1 (char 0)
'NoneType' object is not subscriptable


[(1103, 'SURFNET-NL - SURFnet bv'), (6939, 'HURRICANE')]
[(200070, 'CZNIC-AS2 - CZ.NIC, z.s.p.o.'), (25192, 'CZNIC-AS - CZ.NIC, z.s.p.o.')]


Expecting value: line 1 column 1 (char 0)
'NoneType' object is not subscriptable


[(1103, 'SURFNET-NL - SURFnet bv'), (6939, 'HURRICANE')]
[(3910, 'CENTURYLINK-EUROPE-LEGACY-QWEST'), (3908, 'CENTURYLINK-ASIA-LEGACY-QWEST'), (209, 'CENTURYLINK-US-LEGACY-QWEST')]
[(3910, 'CENTURYLINK-EUROPE-LEGACY-QWEST'), (3908, 'CENTURYLINK-ASIA-LEGACY-QWEST'), (209, 'CENTURYLINK-US-LEGACY-QWEST')]
[(397228, 'ULTRADNS'), (397220, 'ULTRADNS'), (397232, 'ULTRADNS'), (397237, 'ULTRADNS'), (397224, 'ULTRADNS'), (397238, 'ULTRADNS'), (397225, 'ULTRADNS'), (397215, 'ULTRADNS')]
[(1103, 'SURFNET-NL - SURFnet bv'), (6939, 'HURRICANE')]
[(1103, 'SURFNET-NL - SURFnet bv'), (6939, 'HURRICANE')]
[(16589, '-Reserved AS-'), (23393, 'NUCDN')]


Expecting value: line 1 column 1 (char 0)
'NoneType' object is not subscriptable


[(28118, 'ALTICE DOMINICANA S.A.'), (12066, 'ALTICE DOMINICANA S.A.')]
[(1103, 'SURFNET-NL - SURFnet bv'), (6939, 'HURRICANE')]
[(1103, 'SURFNET-NL - SURFnet bv'), (6939, 'HURRICANE')]
[(8717, 'SPECTRUMNET - A1 Bulgaria EAD'), (35141, 'MEGALAN - A1 Bulgaria EAD')]
[(35141, 'MEGALAN - A1 Bulgaria EAD'), (8717, 'SPECTRUMNET - A1 Bulgaria EAD')]


Expecting value: line 1 column 1 (char 0)
'NoneType' object is not subscriptable


[(45090, 'CNNIC-TENCENT-NET-AP Shenzhen Tencent Computer Systems Company Limited'), (132203, 'TENCENT-NET-AP-CN Tencent Building')]
[(8717, 'SPECTRUMNET - A1 Bulgaria EAD'), (35141, 'MEGALAN - A1 Bulgaria EAD')]


In [57]:
unid_rslv

,resolver_address,owner
0,193.62.22.10,JANET - Jisc Services Limited
1,193.62.22.2,JANET - Jisc Services Limited
2,1.1.1.2,CLOUDFLARENET
3,27.114.140.40,DHIRAAGU-MV-AP DHIVEHI RAAJJEYGE GULHUN PLC
4,27.114.138.4,DHIRAAGU-MV-AP DHIVEHI RAAJJEYGE GULHUN PLC
...,...,...
4633,2a0a:e5c0:0:a::a,UNGLEICH-PLACE5 - ungleich glarus ag
4634,2a0a:e5c0:0:a::b,UNGLEICH-PLACE5 - ungleich glarus ag
4635,2001:678:888:69:c45d:2738:c3f2:1878,DANDRI - David Andri Jakobsson
4636,2001:678:888:69:dbce:c9d5:a126:7cf9,DANDRI - David Andri Jakobsson


In [58]:
def pretty_print_asn_owner(asn_owner):
    if asn_owner != None:
        return asn_owner.split(' - ')[0]
    else:
        return 'failed'
#df['holder'].str.split(' - ').str.get(0)

In [59]:
unid_rslv['owner_id'] = unid_rslv.apply(lambda row: pretty_print_asn_owner(row.owner), axis=1)

In [60]:
unid_rslv[unid_rslv.owner_id == 'failed']

,resolver_address,owner,owner_id
364,193.231.252.1,None,failed
365,213.154.124.1,None,failed
569,193.169.198.1,None,failed
585,204.128.136.2,None,failed
667,185.199.216.20,None,failed
703,150.14.0.219,None,failed
704,150.14.0.222,None,failed
736,147.18.0.39,None,failed
777,ff80::2,None,failed
946,2401:4800::123:108:224:6,None,failed


In [131]:
get_asn_from_endpoint('156.154.71.4') #ULTRADNS -> Neustar

'ULTRADNS - NeuStar'

In [132]:
get_asn_from_endpoint('77.88.8.7') #YANDEX -> Yandex

'YANDEX - YANDEX LLC'

In [133]:
get_asn_from_endpoint('64.6.65.6') #VRSN- ->VeriSign

[(396555, 'VRSN-AC50-340 - VeriSign Global Registry Services'), (397208, 'VRSN-AC28 - VeriSign Global Registry Services'), (396547, 'VRSN-AC50-340 - VeriSign Global Registry Services'), (40647, 'VGRS-AC25 - VeriSign Global Registry Services'), (396551, 'VRSN-AC50-340 - VeriSign Global Registry Services')]


'VRSN-AC50-340 - VeriSign Global Registry Services'

In [145]:
get_asn_from_endpoint('149.112.112.112') #QUAD9- ->Quad9

'QUAD9-AS-1 - Quad9'

In [135]:
get_asn_from_endpoint('1.1.1.1') #CLOUDFLARENET -> Cloudflare

'CLOUDFLARENET - Cloudflare'

In [143]:
get_asn_from_endpoint('185.228.169.11') #CIDNOC -> CleanBrowsing

'CIDNOC - Daniel Cid'

In [138]:
get_asn_from_endpoint('8.8.8.8') #GOOGLE -> Google

'GOOGLE - Google LLC'

In [139]:
get_asn_from_endpoint('208.67.222.123') #OPENDNS -> OpenDNS

'OPENDNS - Cisco OpenDNS'

In [147]:
get_asn_from_endpoint('169.239.202.202') #SILENT-GHOST-AS -> OpenNIC

'SILENT-GHOST-AS - Kevin Holly trading as Silent Ghost e.U.'

In [61]:
unid_rslv['centr_resolver_name'] = 'none'

In [62]:
unid_rslv['centr_resolver_name'] = np.where(unid_rslv.owner_id.str.startswith('VRSN-'), 'VeriSign', unid_rslv['centr_resolver_name'])

unid_rslv[unid_rslv.centr_resolver_name == 'VeriSign']




,resolver_address,owner,owner_id,centr_resolver_name


In [63]:
unid_rslv['centr_resolver_name'] = np.where(unid_rslv.owner_id.str.startswith('QUAD9-'), 'Quad9', unid_rslv['centr_resolver_name'])

unid_rslv[unid_rslv.centr_resolver_name == 'Quad9']

,resolver_address,owner,owner_id,centr_resolver_name
403,9.9.9.11,QUAD9-AS-1,QUAD9-AS-1,Quad9
2423,2620:fe::fe:11,QUAD9-AS-1,QUAD9-AS-1,Quad9
3543,2620:fe::12,QUAD9-AS-1,QUAD9-AS-1,Quad9


In [64]:
unid_rslv['centr_resolver_name'] = np.where(unid_rslv.owner_id == 'ULTRADNS', 'Neustar', unid_rslv['centr_resolver_name'])

unid_rslv[unid_rslv.centr_resolver_name == 'Neustar']

,resolver_address,owner,owner_id,centr_resolver_name
2633,156.154.71.10,ULTRADNS,ULTRADNS,Neustar


In [65]:
unid_rslv['centr_resolver_name'] = np.where(unid_rslv.owner_id == 'YANDEX', 'Yandex', unid_rslv['centr_resolver_name'])

unid_rslv[unid_rslv.centr_resolver_name == 'Yandex']

,resolver_address,owner,owner_id,centr_resolver_name
4572,2a02:6bd::feed:ff,YANDEX - YANDEX LLC,YANDEX,Yandex


In [66]:
unid_rslv['centr_resolver_name'] = np.where(unid_rslv.owner_id == 'CLOUDFLARENET', 'Cloudflare', unid_rslv['centr_resolver_name'])

unid_rslv[unid_rslv.centr_resolver_name == 'Cloudflare']

,resolver_address,owner,owner_id,centr_resolver_name
2,1.1.1.2,CLOUDFLARENET,CLOUDFLARENET,Cloudflare
553,1.1.1.3,CLOUDFLARENET,CLOUDFLARENET,Cloudflare
554,1.0.0.3,CLOUDFLARENET,CLOUDFLARENET,Cloudflare
3095,2606:4700:4700::1113,CLOUDFLARENET,CLOUDFLARENET,Cloudflare
3645,1.0.0.2,CLOUDFLARENET,CLOUDFLARENET,Cloudflare
3940,2606:4700:4700:1111::,CLOUDFLARENET,CLOUDFLARENET,Cloudflare
3941,2606:4700:4700:1001::,CLOUDFLARENET,CLOUDFLARENET,Cloudflare
4242,172.64.36.1,CLOUDFLARENET,CLOUDFLARENET,Cloudflare
4243,172.64.36.2,CLOUDFLARENET,CLOUDFLARENET,Cloudflare
4271,2606:4700:4700::1112,CLOUDFLARENET,CLOUDFLARENET,Cloudflare


In [67]:
unid_rslv['centr_resolver_name'] = np.where(unid_rslv.owner_id == 'CIDNOC', 'CleanBrowsing', unid_rslv['centr_resolver_name'])

unid_rslv[unid_rslv.centr_resolver_name == 'CleanBrowsing']

,resolver_address,owner,owner_id,centr_resolver_name


In [68]:
unid_rslv['centr_resolver_name'] = np.where(unid_rslv.owner_id == 'GOOGLE', 'Google', unid_rslv['centr_resolver_name'])

unid_rslv[unid_rslv.centr_resolver_name == 'Google']

,resolver_address,owner,owner_id,centr_resolver_name
1004,8.8.8.4,GOOGLE,GOOGLE,Google
3728,35.246.132.11,GOOGLE,GOOGLE,Google


In [69]:
unid_rslv['centr_resolver_name'] = np.where(unid_rslv.owner_id == 'OPENDNS', 'OpenDNS', unid_rslv['centr_resolver_name'])

unid_rslv[unid_rslv.centr_resolver_name == 'OpenDNS']

,resolver_address,owner,owner_id,centr_resolver_name


In [70]:
unid_rslv['centr_resolver_name'] = np.where(unid_rslv.owner_id == 'SILENT-GHOST-AS', 'OpenNIC', unid_rslv['centr_resolver_name'])

unid_rslv[unid_rslv.centr_resolver_name == 'OpenNIC']

,resolver_address,owner,owner_id,centr_resolver_name


In [71]:

unid_rslv['centr_resolver_name'] = np.where(unid_rslv.owner_id == 'NEXTDNS', 'NextDNS', unid_rslv['centr_resolver_name'])

unid_rslv[unid_rslv.centr_resolver_name == 'NextDNS']

,resolver_address,owner,owner_id,centr_resolver_name
51,45.90.28.102,"NEXTDNS - nextdns, Inc.",NEXTDNS,NextDNS
52,45.90.30.102,"NEXTDNS - nextdns, Inc.",NEXTDNS,NextDNS
159,45.90.28.146,"NEXTDNS - nextdns, Inc.",NEXTDNS,NextDNS
160,45.90.30.146,"NEXTDNS - nextdns, Inc.",NEXTDNS,NextDNS
456,45.90.28.218,"NEXTDNS - nextdns, Inc.",NEXTDNS,NextDNS
457,45.90.30.218,"NEXTDNS - nextdns, Inc.",NEXTDNS,NextDNS
722,45.90.28.234,"NEXTDNS - nextdns, Inc.",NEXTDNS,NextDNS
723,45.90.30.234,"NEXTDNS - nextdns, Inc.",NEXTDNS,NextDNS
2667,45.90.28.112,"NEXTDNS - nextdns, Inc.",NEXTDNS,NextDNS
2668,45.90.30.112,"NEXTDNS - nextdns, Inc.",NEXTDNS,NextDNS


In [72]:
id_rslv = unid_rslv[unid_rslv.centr_resolver_name != 'none'][['resolver_address', 'centr_resolver_name']]

In [73]:
id_rslv

,resolver_address,centr_resolver_name
2,1.1.1.2,Cloudflare
51,45.90.28.102,NextDNS
52,45.90.30.102,NextDNS
159,45.90.28.146,NextDNS
160,45.90.30.146,NextDNS
403,9.9.9.11,Quad9
456,45.90.28.218,NextDNS
457,45.90.30.218,NextDNS
553,1.1.1.3,Cloudflare
554,1.0.0.3,Cloudflare


In [74]:
dff = pd.merge(dff, id_rslv, how='left', on=['resolver_address'])

In [75]:
dff['centr_resolver_name'] = dff['centr_resolver_name'].fillna(value='none')

In [76]:
dff['centr_resolver_name'].value_counts()

none          37358
Cloudflare       55
NextDNS          27
Google            7
Quad9             5
Neustar           1
Yandex            1
Name: centr_resolver_name, dtype: int64

In [77]:
dff['resolver_name'] = np.where((dff['resolver_name'] == 'local')&(dff['centr_resolver_name'] != 'none'), dff['centr_resolver_name'], dff['resolver_name'])

In [78]:
dff[dff.centr_resolver_name != 'none']

,msm_id,prb_id,timestamp,resolver_address,address_family,rt,error,local_resolver,abuf,target_name,rcode,response_type,response_address,ttl,is_global_ip,resolver_name,is_private_ip,centr_resolver_name
6,28317213,1035,1606922981,1.1.1.2,4,56.285,,True,GESBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,142.250.67.14,239,True,Cloudflare,False,Cloudflare
154,28317213,1322,1606922979,45.90.28.102,4,57.38,,True,XJyBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.206.238,176,True,NextDNS,False,NextDNS
155,28317213,1322,1606922979,45.90.30.102,4,64.934,,True,TBaBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.22.142,207,True,NextDNS,False,NextDNS
485,28317213,2371,1606922981,45.90.28.146,4,83.125,,True,wuSBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.17.46,300,True,NextDNS,False,NextDNS
486,28317213,2371,1606922981,45.90.30.146,4,106.515,,True,YcaBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.23.14,300,True,NextDNS,False,NextDNS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36894,28317228,1000916,1606923009,1.1.1.2,4,1.606,,True,eleBgAABAAEAAAAABmdvb2dsZQNjb20AABwAAcAMABwAAQ...,google.com.,NOERROR,AAAA,2607:f8b0:4007:813:0:0:0:200e,99,True,Cloudflare,False,Cloudflare
36895,28317228,1000916,1606923009,2606:4700:4700::1112,6,1.558,,True,FkuBgAABAAEAAAAABmdvb2dsZQNjb20AABwAAcAMABwAAQ...,google.com.,NOERROR,AAAA,2607:f8b0:4007:813:0:0:0:200e,100,True,Cloudflare,False,Cloudflare
37405,28317228,55712,1606923010,1.1.1.2,4,10.68,,True,mgeBgAABAAEAAAAABmdvb2dsZQNjb20AABwAAcAMABwAAQ...,google.com.,NOERROR,AAAA,2607:f8b0:4009:802:0:0:0:200e,133,True,Cloudflare,False,Cloudflare
37406,28317228,55712,1606923010,2606:4700:4700::1112,6,12.081,,True,uPiBgAABAAEAAAAABmdvb2dsZQNjb20AABwAAcAMABwAAQ...,google.com.,NOERROR,AAAA,2607:f8b0:4009:802:0:0:0:200e,123,True,Cloudflare,False,Cloudflare


In [80]:
conn = sqlite3.connect('data/ripe-dns-all-probes-v4v6-extended-resolvers-2020.db')
dff.to_sql('data', con=conn, index=False, if_exists='replace')
conn.close()

In [81]:
df_reduced = dff[['prb_id', 'resolver_name', 'is_global_ip']]

In [82]:
df_reduced = df_reduced.drop_duplicates()

In [83]:
df_reduced.prb_id.nunique()

10624

In [84]:
df_reduced[df_reduced.resolver_name != 'local'].prb_id.nunique()

3007

In [85]:
df_reduced[df_reduced.resolver_name == 'local'].prb_id.nunique()

9253

In [86]:
probes_with_centr_rslv = df_reduced[df_reduced.resolver_name != 'local'].prb_id.unique()

In [87]:
probes_with_local_rslv = df_reduced[df_reduced.resolver_name == 'local'].prb_id.unique()

In [88]:
#probes with a local resolver and centralized one
df_reduced[(df_reduced.prb_id.isin(probes_with_local_rslv)) & (df_reduced.resolver_name != 'local')].prb_id.nunique()

1636

In [89]:
#sanity check
df_reduced[(df_reduced.prb_id.isin(probes_with_centr_rslv)) & (df_reduced.resolver_name == 'local')].prb_id.nunique()

1636

In [90]:
df_reduced['ctr_local'] = np.where(df_reduced['resolver_name'] == 'local', 1, 0)
df_reduced['ctr_centr'] = np.where(df_reduced['resolver_name'] != 'local', 1, 0)

In [91]:
df_reduced['ctr_all'] = 1

In [92]:
df_reduced[(df_reduced.resolver_name == 'local') & (df_reduced.ctr_centr == 1)]

,prb_id,resolver_name,is_global_ip,ctr_local,ctr_centr,ctr_all


In [93]:
df_reduced[(df_reduced.resolver_name != 'local') & (df_reduced.ctr_local == 1)]

,prb_id,resolver_name,is_global_ip,ctr_local,ctr_centr,ctr_all


In [94]:
df_rslv_ctr = df_reduced.groupby('prb_id').agg({'ctr_local':'sum', 'ctr_centr':'sum', 'ctr_all':'sum'}).reset_index()

In [95]:
print('Overall number of probes')
print(df_rslv_ctr.prb_id.count())
print('Mean number of local resolvers')
print(df_rslv_ctr.ctr_local.mean())
print('Median number of local resolvers')
print(df_rslv_ctr.ctr_local.median())
print('Mean number of centralised resolvers')
print(df_rslv_ctr.ctr_centr.mean())
print('Median number of centralised resolvers')
print(df_rslv_ctr.ctr_centr.median())
print('Mean number of all resolvers')
print(df_rslv_ctr.ctr_all.mean())
print('Median number of all resolvers')
print(df_rslv_ctr.ctr_all.median())

Overall number of probes
10624
Mean number of local resolvers
0.9821159638554217
Median number of local resolvers
1.0
Mean number of centralised resolvers
0.39504894578313254
Median number of centralised resolvers
0.0
Mean number of all resolvers
1.3771649096385543
Median number of all resolvers
1.0


In [96]:
print('Number of probes with only local resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].prb_id.count())
print('Mean number of local resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_local.mean())
print('Median number of local resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_local.median())
print('Mean number of centralised resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_centr.mean())
print('Median number of centralised resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_centr.median())
print('Mean number of all resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_all.mean())
print('Median number of all resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_all.median())

Number of probes with only local resolvers
7617
Mean number of local resolvers
1.1463830904555599
Median number of local resolvers
1.0
Mean number of centralised resolvers
0.0
Median number of centralised resolvers
0.0
Mean number of all resolvers
1.1463830904555599
Median number of all resolvers
1.0


In [97]:
#df_rslv_ctr[df_rslv_ctr.ctr_local == 0].prb_id.count()


print('Number of probes with only centralised resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].prb_id.count())
print('Mean number of local resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_local.mean())
print('Median number of local resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_local.median())
print('Mean number of centralised resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_centr.mean())
print('Median number of centralised resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_centr.median())
print('Mean number of all resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_all.mean())
print('Median number of all resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_all.median())

Number of probes with only centralised resolvers
1371
Mean number of local resolvers
0.0
Median number of local resolvers
0.0
Mean number of centralised resolvers
1.314369073668855
Median number of centralised resolvers
1.0
Mean number of all resolvers
1.314369073668855
Median number of all resolvers
1.0


In [98]:
#df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].prb_id.count()

print('Number of probes with centralised and local resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].prb_id.count())
print('Mean number of local resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_local.mean())
print('Median number of local resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_local.median())
print('Mean number of centralised resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_centr.mean())
print('Median number of centralised resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_centr.median())
print('Mean number of all resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_all.mean())
print('Median number of all resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_all.median())

Number of probes with centralised and local resolvers
1636
Mean number of local resolvers
1.0403422982885087
Median number of local resolvers
1.0
Mean number of centralised resolvers
1.4639364303178484
Median number of centralised resolvers
1.0
Mean number of all resolvers
2.504278728606357
Median number of all resolvers
3.0


In [205]:
measurements4

array([22001500, 22001501, 22001502, 22001503, 22001504])

In [206]:
measurements6

array([22001505, 22001506])

In [207]:
dff[dff.msm_id.isin(measurements4)]

,msm_id,prb_id,timestamp,resolver_address,address_family,rt,error,local_resolver,abuf,target_name,rcode,response_type,response_address,ttl,is_global_ip,resolver_name,is_private_ip,centr_resolver_name
0,22001500,627,1560433280,193.162.153.164,4,52.577,,True,sPKBgAABAAEABAAIBmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.207.206,112,True,local,False,none
1,22001500,627,1560433280,194.239.134.83,4,53.224,,True,/KWBgAABAAEABAAIBmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.21.174,279,True,local,False,none
2,22001500,10001,1560433694,fd00::eadf:70ff:fea3:7a26,6,7.765,,True,y7yBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.168.238,85,False,local,True,none
3,22001500,10001,1560433694,192.168.178.1,4,1.259,,True,08KBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.168.238,84,False,local,True,none
4,22001500,10006,1560433693,8.8.8.8,4,28.205,,True,QpqBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.169.46,264,True,Google,False,none
5,22001500,10007,1560433695,75.75.75.75,4,14.097,,True,sPKBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.6.78,247,True,local,False,none
6,22001500,10007,1560433695,2001:558:feed::1,6,13.821,,True,VkGBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.194.174,273,True,local,False,none
7,22001500,10007,1560433695,75.75.76.76,4,31.651,,True,/KWBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,216.58.195.78,240,True,local,False,none
8,22001500,10009,1560433694,192.168.2.1,4,10.52,,True,rqaBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.5.110,242,False,local,True,none
9,22001500,10009,1560433694,2601:646:c000:d16c::1,6,0.665,,True,UQKBgAABAAEAAAAABmdvb2dsZQNjb20AAAEAAcAMAAEAAQ...,google.com.,NOERROR,A,172.217.5.110,241,True,local,False,none


In [23]:
df_reduced

,prb_id,resolver_name,is_global_ip,ctr_local,ctr_centr,ctr_all
0,627,local,1,1,0,1
2,10001,local,0,1,0,1
4,10006,Google,1,0,1,1
5,10007,local,1,1,0,1
8,10009,local,0,1,0,1
9,10009,local,1,1,0,1
10,10012,local,0,1,0,1
12,10015,local,0,1,0,1
14,10016,local,1,1,0,1
17,10019,local,0,1,0,1


In [117]:
db = sqlite3.connect("data/ripe-dns-all-probes-v4v6-extended-resolvers-2020.db")
dff[['msm_id', 'prb_id', 'timestamp', 'resolver_address', 'address_family', 'rt', 'error', 'local_resolver', 'target_name', 'rcode', 'response_type', 'response_address', 'ttl', 'is_global_ip', 'resolver_name']].to_sql('data', db, if_exists='replace', index=False)
db.close()